In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
from google.colab import files
df = pd.read_csv("anomaly_label.csv")
print(df.shape)
df.head(10)

(575061, 2)


,BlockId,Label
0,blk_-1608999687919862906,Normal
1,blk_7503483334202473044,Normal
2,blk_-3544583377289625738,Anomaly
3,blk_-9073992586687739851,Normal
4,blk_7854771516489510256,Normal
5,blk_1717858812220360316,Normal
6,blk_-2519617320378473615,Normal
7,blk_7063315473424667801,Normal
8,blk_8586544123689943463,Normal
9,blk_2765344736980045501,Normal


In [ ]:
from google.colab import files
df_HDFS= pd.read_csv("HDFS_100k.log_structured.csv")
print(df_HDFS.shape)
df_HDFS.head(10)

(104815, 9)


,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate
0,1,81109,203518,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>
1,2,81109,203518,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...,E22,BLOCK* NameSystem.allocateBlock:<*>
2,3,81109,203519,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>
3,4,81109,203519,145,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>
4,5,81109,203519,145,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_-1608999687919...,E11,PacketResponder <*> for block <*> terminating
5,6,81109,203519,145,INFO,dfs.DataNode$PacketResponder,PacketResponder 2 for block blk_-1608999687919...,E11,PacketResponder <*> for block <*> terminating
6,7,81109,203519,145,INFO,dfs.DataNode$PacketResponder,Received block blk_-1608999687919862906 of siz...,E9,Received block <*> of size <*> from /<*>
7,8,81109,203519,145,INFO,dfs.DataNode$PacketResponder,Received block blk_-1608999687919862906 of siz...,E9,Received block <*> of size <*> from /<*>
8,9,81109,203519,147,INFO,dfs.DataNode$PacketResponder,PacketResponder 0 for block blk_-1608999687919...,E11,PacketResponder <*> for block <*> terminating
9,10,81109,203519,147,INFO,dfs.DataNode$PacketResponder,Received block blk_-1608999687919862906 of siz...,E9,Received block <*> of size <*> from /<*>


In [ ]:
df_HDFS.iloc[0].Content

'Receiving block blk_-1608999687919862906 src: /10.250.19.102:54106 dest: /10.250.19.102:50010'

In [ ]:
df_HDFS['BlockID']=None
df_HDFS

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate,BlockID
0,1,81109,203518,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>,None
1,2,81109,203518,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...,E22,BLOCK* NameSystem.allocateBlock:<*>,None
2,3,81109,203519,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>,None
3,4,81109,203519,145,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>,None
4,5,81109,203519,145,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_-1608999687919...,E11,PacketResponder <*> for block <*> terminating,None
...,...,...,...,...,...,...,...,...,...,...
104810,104811,81109,210122,13,INFO,dfs.DataBlockScanner,Verification succeeded for blk_-87387097785867...,E2,Verification succeeded for <*>,None
104811,104812,81109,210124,13,INFO,dfs.DataBlockScanner,Verification succeeded for blk_642047611142564...,E2,Verification succeeded for <*>,None
104812,104813,81109,210126,13,INFO,dfs.DataBlockScanner,Verification succeeded for blk_-23823897510323...,E2,Verification succeeded for <*>,None
104813,104814,81109,210126,13,INFO,dfs.DataBlockScanner,Verification succeeded for blk_485603173001003...,E2,Verification succeeded for <*>,None


In [ ]:
# Extract block IDs from 'Content'
import re

# Define the regex pattern to match 'blk_' followed by numbers
pattern = r'blk_-?\d+'
blockIDs=[]
# Use re.search to find the first match in the log message
for i in range(0,df_HDFS.shape[0]):
  match = re.search(pattern, df_HDFS.iloc[i].Content)
  if match:
    blk = match.group()
    blockIDs.append(blk)
    df_HDFS.at[i, 'BlockID'] = blk
  i+=1

In [ ]:
len(blockIDs)

104815

In [ ]:
df_HDFS.head(15)

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate,BlockID
0,1,81109,203518,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>,blk_-1608999687919862906
1,2,81109,203518,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...,E22,BLOCK* NameSystem.allocateBlock:<*>,blk_-1608999687919862906
2,3,81109,203519,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>,blk_-1608999687919862906
3,4,81109,203519,145,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>,blk_-1608999687919862906
4,5,81109,203519,145,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_-1608999687919...,E11,PacketResponder <*> for block <*> terminating,blk_-1608999687919862906
5,6,81109,203519,145,INFO,dfs.DataNode$PacketResponder,PacketResponder 2 for block blk_-1608999687919...,E11,PacketResponder <*> for block <*> terminating,blk_-1608999687919862906
6,7,81109,203519,145,INFO,dfs.DataNode$PacketResponder,Received block blk_-1608999687919862906 of siz...,E9,Received block <*> of size <*> from /<*>,blk_-1608999687919862906
7,8,81109,203519,145,INFO,dfs.DataNode$PacketResponder,Received block blk_-1608999687919862906 of siz...,E9,Received block <*> of size <*> from /<*>,blk_-1608999687919862906
8,9,81109,203519,147,INFO,dfs.DataNode$PacketResponder,PacketResponder 0 for block blk_-1608999687919...,E11,PacketResponder <*> for block <*> terminating,blk_-1608999687919862906
9,10,81109,203519,147,INFO,dfs.DataNode$PacketResponder,Received block blk_-1608999687919862906 of siz...,E9,Received block <*> of size <*> from /<*>,blk_-1608999687919862906


In [ ]:
df = df.rename(columns={'BlockId': 'BlockID'})
df

,BlockID,Label
0,blk_-1608999687919862906,Normal
1,blk_7503483334202473044,Normal
2,blk_-3544583377289625738,Anomaly
3,blk_-9073992586687739851,Normal
4,blk_7854771516489510256,Normal
...,...,...
575056,blk_1019720114020043203,Normal
575057,blk_-2683116845478050414,Normal
575058,blk_5595059397348477632,Normal
575059,blk_1513937873877967730,Normal


In [ ]:
# Merge the DataFrames based on multiple keys
merged_df = df_HDFS.merge(df,on='BlockID', how='left')

# Display the updated df1
merged_df.head(20)

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate,BlockID,Label
0,1,81109,203518,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>,blk_-1608999687919862906,Normal
1,2,81109,203518,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...,E22,BLOCK* NameSystem.allocateBlock:<*>,blk_-1608999687919862906,Normal
2,3,81109,203519,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>,blk_-1608999687919862906,Normal
3,4,81109,203519,145,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>,blk_-1608999687919862906,Normal
4,5,81109,203519,145,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_-1608999687919...,E11,PacketResponder <*> for block <*> terminating,blk_-1608999687919862906,Normal
5,6,81109,203519,145,INFO,dfs.DataNode$PacketResponder,PacketResponder 2 for block blk_-1608999687919...,E11,PacketResponder <*> for block <*> terminating,blk_-1608999687919862906,Normal
6,7,81109,203519,145,INFO,dfs.DataNode$PacketResponder,Received block blk_-1608999687919862906 of siz...,E9,Received block <*> of size <*> from /<*>,blk_-1608999687919862906,Normal
7,8,81109,203519,145,INFO,dfs.DataNode$PacketResponder,Received block blk_-1608999687919862906 of siz...,E9,Received block <*> of size <*> from /<*>,blk_-1608999687919862906,Normal
8,9,81109,203519,147,INFO,dfs.DataNode$PacketResponder,PacketResponder 0 for block blk_-1608999687919...,E11,PacketResponder <*> for block <*> terminating,blk_-1608999687919862906,Normal
9,10,81109,203519,147,INFO,dfs.DataNode$PacketResponder,Received block blk_-1608999687919862906 of siz...,E9,Received block <*> of size <*> from /<*>,blk_-1608999687919862906,Normal


In [ ]:
merged_df.shape

(104815, 11)

In [ ]:
merged_df.to_csv('/content/drive/My Drive/merged_HDFS.csv', index=False)

In [ ]:
from google.colab import files

# Specify the path to the CSV file in Google Drive
google_drive_path = '/content/drive/My Drive/merged_HDFS.csv'

# Download the file to your local PC
files.download(google_drive_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
merged_df['Label'].to_csv('/content/drive/My Drive/anomaly2.csv')
from google.colab import files

# Specify the path to the CSV file in Google Drive
google_drive_path = '/content/drive/My Drive/anomaly2.csv'

# Download the file to your local PC
files.download(google_drive_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**PREPROCESSING**

In [ ]:
import dataloader, preprocessing
import numpy as np
import pandas as pd

# Load dataset
parsed_log='/content/HDFS_100k.log_structured.csv'
labels='/content/anomaly_label.csv'
# No train/test split (done in modelling notebook)
(X_train, y_train),(X_test, y_test)=dataloader.load_HDFS(parsed_log,label_file=labels,window='session',train_ratio=1,split_type='uniform',save_csv=True)

# Define feature extractor
feature_extractor=preprocessing.FeatureExtractor()

# Perform feature extraction on training set
X_train=feature_extractor.fit_transform(X_train,term_weighting=None,normalization=None)

====== Input data summary ======
Loading /content/HDFS_100k.log_structured.csv
313 0
Total: 7940 instances, 313 anomaly, 7627 normal
Train: 7940 instances, 313 anomaly, 7627 normal
Test: 0 instances, 0 anomaly, 0 normal

====== Transformed train data summary ======
Train data shape: 7940-by-19



In [ ]:
print(feature_extractor.term_weighting)
print(feature_extractor.normalization)
print(X_train.shape)
print(y_train.shape)
print(y_test.shape)

None
None
(7940, 19)
(7940,)
(0,)


In [ ]:
# Convert to dataframe
data=pd.DataFrame(X_train)
data['anomaly_label']=y_train
print(data.shape)
data.head()

(7940, 20)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,anomaly_label
0,1.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


**MODEL**

In [ ]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)

In [ ]:
X_train,X_test,y_train,y_test=model_selection.train_test_split(data.iloc[:,:-1],data.loc[:,'anomaly_label'],test_size=0.15,random_state=586)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(6749, 19)
(6749,)
(1191, 19)
(1191,)


In [ ]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
3224,1.000000,3.000000,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1947,1.000000,3.000000,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3214,1.000000,3.000000,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4923,1.000000,3.000000,3.000000,3.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7364,1.000000,3.000000,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
print('Train percent anamolies:',str(np.round(y_train.mean(),4)))
print('Test percent anamolies:',str(np.round(y_test.mean(),4)))

Train percent anamolies: 0.0381
Test percent anamolies: 0.047


**Standardization**

In [ ]:
## Perform standardization

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

# Fit StandardScaler on training set
scaler=preprocessing.StandardScaler().fit(X_train)

# Apply standardization to train/test sets
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)

# Examine mean/std of first 5
print('Train mean:',X_train_scaled.mean(axis=0)[0:5])
print('Test mean:',X_test_scaled.mean(axis=0)[0:5])

print('Train std:',X_train_scaled.std(axis=0)[0:5])
print('Test std:',X_test_scaled.std(axis=0)[0:5])

Train mean: [ 0.00000000e+00 -5.08771340e-16 -2.26354554e-16 -3.48743934e-16
  3.95857266e-16]
Test mean: [ 0.         -0.03433538 -0.03294591 -0.02875407 -0.02710955]
Train std: [0. 1. 1. 1. 1.]
Test std: [0.         0.96070908 1.06689363 1.10410039 1.10310052]


**PCA (Dimensionality Reduction)**

In [ ]:
# Perform PCA on scaled data
pca=PCA()
pca.fit(X_train_scaled)

PCA()

In [ ]:
# Examine cumulative proportion of variance explained
print(pca.explained_variance_ratio_.cumsum())

num_components=np.argmax(pca.explained_variance_ratio_.cumsum()>=0.99)
print('# Components to explain 99% of variance:',num_components)

[0.30403155 0.54088228 0.72789931 0.78528852 0.84116729 0.89674369
 0.95165927 0.97674781 0.98977825 0.99835603 0.99962678 1.
 1.         1.         1.         1.         1.         1.
 1.        ]
# Components to explain 99% of variance: 9


In [ ]:
# Get component scores for train/test sets
X_train_scores=pca.transform(X_train_scaled)[:,:num_components]
X_test_scores=pca.transform(X_test_scaled)[:,:num_components]
print(X_train_scores.shape)
print(X_test_scores.shape)

(6749, 9)
(1191, 9)


**Weighted binary cross entropy loss function**

In [ ]:
def weighted_bce(y_true,y_pred):
    sample_weights=(y_true.values==1).astype(float)
    sample_weights=sample_weights*(1/np.mean(y_true))
    sample_weights=np.where(sample_weights==0,1,sample_weights)
    return metrics.log_loss(y_true=y_true,y_pred=y_pred,sample_weight=sample_weights)

**Perform logistic regression on component scores**

In [ ]:
# Define dictionary of metrics for CV
eval_metrics={'weighted_bce':metrics.make_scorer(weighted_bce),'accuracy':metrics.make_scorer(metrics.accuracy_score),'recall':metrics.make_scorer(metrics.recall_score),'precision':metrics.make_scorer(metrics.precision_score)}

# Get 5-fold CV estimates for logistic regression model
logistic_regressor_cv=model_selection.cross_validate(LogisticRegression(max_iter=1000,n_jobs=-1),X=X_train_scores,y=y_train,scoring=eval_metrics,n_jobs=-1,verbose=1)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.3s finished


In [ ]:
logistic_regressor_cv

{'fit_time': array([0.06715631, 0.10877943, 0.09574294, 0.07148695, 0.05394626]),
 'score_time': array([0.02180243, 0.01776075, 0.01402664, 0.02791929, 0.01251936]),
 'test_weighted_bce': array([10.45860633,  9.724655  ,  7.42096774,  8.14133822, 11.17912078]),
 'test_accuracy': array([0.97777778, 0.98      , 0.98444444, 0.98222222, 0.97627872]),
 'test_recall': array([0.43137255, 0.47058824, 0.59615385, 0.55769231, 0.39215686]),
 'test_precision': array([0.95652174, 1.        , 1.        , 0.96666667, 0.95238095])}

In [ ]:
# Get avg error metrics from folds
{key:np.round(np.mean(val),5) for key,val in logistic_regressor_cv.items()}

{'fit_time': 0.07942,
 'score_time': 0.01881,
 'test_weighted_bce': 9.38494,
 'test_accuracy': 0.98014,
 'test_recall': 0.48959,
 'test_precision': 0.97511}

**QDA**

In [ ]:
qda_model=QuadraticDiscriminantAnalysis()
# Get 5-fold CV estimates for QDA
qda_model_cv=model_selection.cross_validate(qda_model,X=X_train_scores,y=y_train,scoring=eval_metrics,n_jobs=-1,verbose=1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.3s finished


In [ ]:
# Get avg error metrics from folds
{key:np.round(np.mean(val),5) for key,val in qda_model_cv.items()}

{'fit_time': 0.0299,
 'score_time': 0.0388,
 'test_weighted_bce': 11.2171,
 'test_accuracy': 0.78029,
 'test_recall': 0.59374,
 'test_precision': 0.64516}

**KNN**

In [ ]:
# Create KNN model with default params
knn_model=KNeighborsClassifier(n_jobs=-1)

In [ ]:
# Get 5-fold CV estimates for default KNN model

# Use 10% of training set (computationally expensive)
knn_model_cv=model_selection.cross_validate(knn_model,X=X_train_scores[:int(len(X_train_scores)*0.1),:],y=y_train[:int(len(X_train_scores)*0.1)],scoring=eval_metrics,n_jobs=-1,verbose=1)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


{'fit_time': 0.00249,
 'score_time': 0.06268,
 'test_weighted_bce': 13.26217,
 'test_accuracy': 0.97033,
 'test_recall': 0.28,
 'test_precision': 0.75}

In [ ]:
# Get avg error metrics from folds
{key:np.round(np.mean(val),5) for key,val in knn_model_cv.items()}

{'fit_time': 0.00249,
 'score_time': 0.06268,
 'test_weighted_bce': 13.26217,
 'test_accuracy': 0.97033,
 'test_recall': 0.28,
 'test_precision': 0.75}

In [ ]:
# Tune k hyperparameter
k_vals=[]
knn_cv_weighted_bce=[]
counter=0
max_k=50
for i in np.arange(1,max_k+1):
    counter+=1
    k_vals.append(i)
    # Define KNN model
    knn_model_tuner=KNeighborsClassifier(n_neighbors=i,n_jobs=-1)
    # Get CV estimate of weighted binary cross entropy loss
    knn_model_tuner_cv=model_selection.cross_validate(knn_model_tuner,X=X_train_scores[:int(len(X_train_scores)*0.1),:],y=y_train[:int(len(X_train_scores)*0.1)],scoring=eval_metrics,n_jobs=-1)
    knn_cv_weighted_bce.append(np.mean(knn_model_tuner_cv['test_weighted_bce']))
    print('Performed iteration '+str(counter)+' of '+str(max_k))
knn_grid_results=pd.DataFrame({'k_val':k_vals,'cv_weighted_bce':knn_cv_weighted_bce})

Performed iteration 1 of 50
Performed iteration 2 of 50
Performed iteration 3 of 50
Performed iteration 4 of 50
Performed iteration 5 of 50
Performed iteration 6 of 50
Performed iteration 7 of 50
Performed iteration 8 of 50
Performed iteration 9 of 50
Performed iteration 10 of 50
Performed iteration 11 of 50
Performed iteration 12 of 50
Performed iteration 13 of 50
Performed iteration 14 of 50
Performed iteration 15 of 50
Performed iteration 16 of 50
Performed iteration 17 of 50
Performed iteration 18 of 50
Performed iteration 19 of 50
Performed iteration 20 of 50
Performed iteration 21 of 50
Performed iteration 22 of 50
Performed iteration 23 of 50
Performed iteration 24 of 50
Performed iteration 25 of 50
Performed iteration 26 of 50
Performed iteration 27 of 50
Performed iteration 28 of 50
Performed iteration 29 of 50
Performed iteration 30 of 50
Performed iteration 31 of 50
Performed iteration 32 of 50
Performed iteration 33 of 50
Performed iteration 34 of 50
Performed iteration 35 

In [ ]:
knn_grid_results.sort_values('cv_weighted_bce').head(10)

,k_val,cv_weighted_bce
0,1,11.178841
1,2,11.178841
2,3,11.178841
5,6,13.234864
6,7,13.234864
7,8,13.234864
8,9,13.234864
3,4,13.262170
4,5,13.262170
12,13,16.921147


In [ ]:
# Create KNN model with tuned k param
tuned_knn_model=KNeighborsClassifier(n_neighbors=3,n_jobs=-1)

In [ ]:
# Get 5-fold CV estimates for default KNN model on whole dataset
tuned_knn_model_cv=model_selection.cross_validate(tuned_knn_model,X=X_train_scores,y=y_train,scoring=eval_metrics,n_jobs=-1,verbose=1)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.6s finished


In [ ]:
# Get avg error metrics from folds
{key:np.round(np.mean(val),5) for key,val in tuned_knn_model_cv.items()}

{'fit_time': 0.04524,
 'score_time': 0.81705,
 'test_weighted_bce': 9.31698,
 'test_accuracy': 0.98014,
 'test_recall': 0.49344,
 'test_precision': 0.96736}

In [ ]:
tuned_knn_model.fit(X_train,y_train)
y_pred = tuned_knn_model.predict(X_test)

In [ ]:
print(y_pred[0:300])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]


In [ ]:
print(y_test[75:90])

3134    0
2750    0
3994    0
6350    0
537     0
2244    1
6731    0
5828    0
734     1
5897    0
1732    0
2145    0
2831    0
801     0
3538    0
Name: anomaly_label, dtype: int64


In [ ]:
import torch
torch.save(tuned_knn_model, '/content/drive/MyDrive/Saved Models/HDFS_KNN.h5')
from google.colab import files
files.download('/content/drive/MyDrive/Saved Models/HDFS_KNN.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Random Forest**

In [ ]:
# Create random forest with default params
random_forest=RandomForestClassifier(n_jobs=-1,random_state=586)

In [ ]:
# Get 5-fold CV estimates for default random forest
random_forest_cv=model_selection.cross_validate(random_forest,X=X_train_scores,y=y_train,scoring=eval_metrics,n_jobs=-1,verbose=1)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.5s finished


In [ ]:
# Get avg error metrics from folds
{key:np.round(np.mean(val),5) for key,val in random_forest_cv.items()}

{'fit_time': 0.44877,
 'score_time': 0.05713,
 'test_weighted_bce': 9.31698,
 'test_accuracy': 0.98014,
 'test_recall': 0.49344,
 'test_precision': 0.96779}

In [ ]:
## Tune random forest hyperparameters using OOB weighted binary cross entropy loss
rf_params={'criterion':['gini','entropy'],
           'max_features':['auto','sqrt','log2'],
           'max_depth':[30,40,50,60,None]}

In [ ]:
# Implement random grid search
counter=0
num_iter=15
rf_grid_results=[]
for i in range(num_iter):
    counter+=1
    # Select random parameter set
    param_choice={key:np.random.choice(val) for key,val in rf_params.items()}
    # Define model with param choices
    random_forest_tuner=RandomForestClassifier(
                           criterion=param_choice['criterion'],
                           max_features=param_choice['max_features'],
                           max_depth=param_choice['max_depth'],
                           oob_score=True,
                           n_jobs=-1,
                           verbose=1,
                           random_state=586)
    # Fit model
    random_forest_tuner.fit(X_train_scores,y_train)
    # Compute OOB binary weighted cross entropy loss
    param_choice['oob_weighted_bce']=weighted_bce(y_train,random_forest_tuner.oob_decision_function_[:, 1])
    rf_grid_results.append(param_choice)
    print('Performed iteration '+str(counter)+' of '+str(num_iter))
rf_grid_results=pd.DataFrame(rf_grid_results)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.4s finished
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


Performed iteration 1 of 15


[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.3s finished


Performed iteration 2 of 15


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.3s finished


Performed iteration 14 of 15


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.1s


Performed iteration 15 of 15


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.3s finished


In [ ]:
rf_grid_results.sort_values('oob_weighted_bce')

,criterion,max_features,max_depth,oob_weighted_bce
0,entropy,auto,60.000000,1.026452
2,entropy,auto,40.000000,1.026452
4,entropy,sqrt,60.000000,1.026452
9,entropy,log2,50.000000,1.026452
10,entropy,auto,NaN,1.026452
11,entropy,auto,40.000000,1.026452
12,entropy,sqrt,50.000000,1.026452
1,gini,auto,50.000000,1.026954
3,gini,log2,30.000000,1.026954
5,gini,sqrt,NaN,1.026954


In [ ]:
# Fit random forest using tuned hyperparameters
best_rf_params=rf_grid_results.sort_values('oob_weighted_bce').iloc[0,:]
tuned_random_forest=RandomForestClassifier(criterion=best_rf_params['criterion'],
                                           max_features=best_rf_params['max_features'],
                                           max_depth=int(best_rf_params['max_depth']),
                                           n_jobs=-1,
                                           random_state=586)

In [ ]:
# Get 5-fold CV estimates for tuned random forest
tuned_random_forest_cv=model_selection.cross_validate(tuned_random_forest,X=X_train_scores,y=y_train,scoring=eval_metrics,n_jobs=-1,verbose=1)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.7s finished


In [ ]:
# Get avg error metrics from folds
{key:np.round(np.mean(val),5) for key,val in tuned_random_forest_cv.items()}

{'fit_time': 0.87565,
 'score_time': 0.08145,
 'test_weighted_bce': 9.31698,
 'test_accuracy': 0.98014,
 'test_recall': 0.49344,
 'test_precision': 0.96779}

**Gradient boosting classifier**

In [ ]:
# Create gradient boosting classifier with default params
gradient_boosting=GradientBoostingClassifier(random_state=586)

In [ ]:
# Get 5-fold CV estimates for gradient boosting classifier
gradient_boosting_cv=model_selection.cross_validate(gradient_boosting,X=X_train_scores,y=y_train,scoring=eval_metrics,n_jobs=-1,verbose=1)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    4.9s finished


In [ ]:
# Get avg error metrics from folds
{key:np.round(np.mean(val),5) for key,val in gradient_boosting_cv.items()}

{'fit_time': 0.72248,
 'score_time': 0.02029,
 'test_weighted_bce': 9.31698,
 'test_accuracy': 0.98014,
 'test_recall': 0.49344,
 'test_precision': 0.96779}

In [ ]:
## Tune gradient boosting classifier hyperparameters
gb_params={'n_estimators':[100,200,300,400,500],
           'subsample':[0.4,0.6,0.8,1],
           'max_depth':[4,6,8,10]}

In [ ]:
# Implement random grid search
# Using 10% sample of dataset -- too computationally expensive otherwise
counter=0
num_iter=15
gb_grid_results=[]
for i in range(num_iter):
    counter+=1
    # Select random parameter set
    param_choice={key:np.random.choice(val) for key,val in gb_params.items()}
    # Define model with param choices
    gradient_boosting_tuner=GradientBoostingClassifier(
                           n_estimators=param_choice['n_estimators'],
                           subsample=param_choice['subsample'],
                           max_depth=param_choice['max_depth'],
                           learning_rate=10/param_choice['n_estimators'],
                           random_state=586)
    # Get CV estimate of weighted binary cross entropy loss
    gradient_boosting_tuner_cv=model_selection.cross_validate(gradient_boosting_tuner,X=X_train_scores[:int(len(X_train_scores)*0.1),:],y=y_train[:int(len(X_train_scores)*0.1)],scoring=eval_metrics,n_jobs=-1,verbose=1)
    param_choice['learning_rate']=10/param_choice['n_estimators']
    param_choice['cv_weighted_bce']=np.mean(gradient_boosting_tuner_cv['test_weighted_bce'])
    gb_grid_results.append(param_choice)
    print('Performed iteration '+str(counter)+' of '+str(num_iter))
gb_grid_results=pd.DataFrame(gb_grid_results)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Performed iteration 1 of 15


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    4.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Performed iteration 2 of 15


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    5.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Performed iteration 13 of 15


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Performed iteration 14 of 15
Performed iteration 15 of 15


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.3s finished


In [ ]:
gb_grid_results.sort_values('cv_weighted_bce')

,n_estimators,subsample,max_depth,learning_rate,cv_weighted_bce
0,100,0.800000,8,0.100000,11.178841
1,200,0.400000,10,0.050000,11.178841
2,400,1.000000,4,0.025000,11.178841
3,200,0.600000,8,0.050000,11.178841
4,200,0.800000,10,0.050000,11.178841
5,200,0.400000,4,0.050000,11.178841
6,300,1.000000,6,0.033333,11.178841
7,100,0.600000,8,0.100000,11.178841
8,300,0.800000,4,0.033333,11.178841
9,100,0.800000,8,0.100000,11.178841


In [ ]:
best_gb_params=gb_grid_results.sort_values('cv_weighted_bce').iloc[0,:]

In [ ]:
gb_lr_params={'learning_rate':[0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15]}

In [ ]:
# Implement exhaustive grid search to further tune learning rate
counter=0
gb_lr_grid_results=[]
for i in range(len(gb_lr_params['learning_rate'])):
    counter+=1
    # Select parameter
    param_choice={key:val[i] for key,val in gb_lr_params.items()}
    # Define model with param choices
    gradient_boosting_lr_tuner=GradientBoostingClassifier(
                           n_estimators=int(best_gb_params['n_estimators']),
                           subsample=best_gb_params['subsample'],
                           max_depth=int(best_gb_params['max_depth']),
                           learning_rate=param_choice['learning_rate'],
                           random_state=586)
    # Get CV estimate of weighted binary cross entropy loss
    gradient_boosting_lr_tuner_cv=model_selection.cross_validate(gradient_boosting_lr_tuner,X=X_train_scores[:int(len(X_train_scores)*0.1),:],y=y_train[:int(len(X_train_scores)*0.1)],scoring=eval_metrics,n_jobs=-1,verbose=1)
    param_choice['cv_weighted_bce']=np.mean(gradient_boosting_lr_tuner_cv['test_weighted_bce'])
    gb_lr_grid_results.append(param_choice)
    print('Performed iteration '+str(counter)+' of '+str(len(gb_lr_params['learning_rate'])))
gb_lr_grid_results=pd.DataFrame(gb_lr_grid_results)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Performed iteration 1 of 11


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Performed iteration 2 of 11


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Performed iteration 9 of 11


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Performed iteration 10 of 11
Performed iteration 11 of 11


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.0s finished


In [ ]:
gb_lr_grid_results.sort_values('cv_weighted_bce')

,learning_rate,cv_weighted_bce
0,0.050000,11.178841
1,0.060000,11.178841
2,0.070000,11.178841
3,0.080000,11.178841
4,0.090000,11.178841
5,0.100000,11.178841
6,0.110000,11.178841
7,0.120000,11.178841
8,0.130000,11.178841
9,0.140000,11.178841


In [ ]:
# Create gradient boosting classifier with tuned params
tuned_gradient_boosting=GradientBoostingClassifier(
                                                  n_estimators=int(best_gb_params['n_estimators']),
                                                  subsample=best_gb_params['subsample'],
                                                  max_depth=int(best_gb_params['max_depth']),
                                                  learning_rate=gb_lr_grid_results.sort_values('cv_weighted_bce').iloc[0]['learning_rate'],
                                                  random_state=586)

In [ ]:
# Get 5-fold CV estimates for tuned gradient boosting classifier
tuned_gradient_boosting_cv=model_selection.cross_validate(tuned_gradient_boosting,X=X_train_scores,y=y_train,scoring=eval_metrics,n_jobs=-1,verbose=1)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.1s finished


In [ ]:
# Get avg error metrics from folds
{key:np.round(np.mean(val),5) for key,val in tuned_gradient_boosting_cv.items()}

{'fit_time': 1.09199,
 'score_time': 0.02465,
 'test_weighted_bce': 9.31698,
 'test_accuracy': 0.98014,
 'test_recall': 0.49344,
 'test_precision': 0.96779}

In [ ]:
tuned_gradient_boosting.fit(X_train,y_train)
y_pred=tuned_gradient_boosting.predict(X_test)
print(y_pred[0:300])

AttributeError: ignored

**Comparison of results**

In [ ]:
all_results=[]
for model in [logistic_regressor_cv,qda_model_cv,knn_model_cv,tuned_knn_model_cv,random_forest_cv,tuned_random_forest_cv,gradient_boosting_cv,tuned_gradient_boosting_cv]:
    all_results.append({key:np.round(np.mean(val),5) for key,val in model.items()})
all_results_df=pd.DataFrame(all_results)
all_results_df['test_f1_score']=(2*all_results_df['test_precision']*all_results_df['test_recall'])/(all_results_df['test_precision']+all_results_df['test_recall'])
all_results_df['model']=['Logistic Regression','QDA','KNN','Tuned KNN','Random Forest','Tuned Random Forest','Gradient Boosting','Tuned Gradient Boosting']
all_results_df=all_results_df.rename(columns={'fit_time':'Avg Fit Time','score_time':'Avg Score Time','test_weighted_bce':'CV Binary Cross Entropy Loss','test_accuracy':'CV Accuracy','test_recall':'CV Recall','test_precision':'CV Precision','test_f1_score':'CV F1 Score','model':'Model'})
all_results_df=all_results_df.sort_values('CV F1 Score',ascending=False)[['Model','CV F1 Score','CV Binary Cross Entropy Loss','CV Accuracy','CV Recall','CV Precision','Avg Fit Time','Avg Score Time']]
all_results_df

,Model,CV F1 Score,CV Binary Cross Entropy Loss,CV Accuracy,CV Recall,CV Precision,Avg Fit Time,Avg Score Time
4,Random Forest,0.653622,9.316980,0.980140,0.493440,0.967790,0.448770,0.057130
5,Tuned Random Forest,0.653622,9.316980,0.980140,0.493440,0.967790,0.875650,0.081450
6,Gradient Boosting,0.653622,9.316980,0.980140,0.493440,0.967790,0.722480,0.020290
7,Tuned Gradient Boosting,0.653622,9.316980,0.980140,0.493440,0.967790,1.091990,0.024650
3,Tuned KNN,0.653524,9.316980,0.980140,0.493440,0.967360,0.045240,0.817050
0,Logistic Regression,0.651880,9.384940,0.980140,0.489590,0.975110,0.079420,0.018810
1,QDA,0.618383,11.217100,0.780290,0.593740,0.645160,0.029900,0.038800
2,KNN,0.407767,13.262170,0.970330,0.280000,0.750000,0.002490,0.062680
